1. 데이터는 pandas의 read_csv() 메서드를 이용하여 winequality-red.csv 파일로부터 읽으면 됩니다.
2. 데이터 전처리 시 PolyFeatures(2차항까지) StandardScaler를 사용하십시오.
3. LogisticRegression을 이용해 와인의 품질을 예측하는 모델 객체를 생성하십시오.
4. GridSearchCV를 사용하여 교차검증을 하되, 최적의 모델을 찾기 위해 penalty, C의 값을 탐색하십시오.
5. 여기서 penalty는 L1, L2, elasticnet을 사용하고, elasticnet에 대해서는 l1_ratio 값을 함께 탐색해야 합니다.
6. 위의 세 가지 규제를 모두 수용하기 위해서 solver는 'saga'를 선택해야 하며, tol=0.001로 설정합니다.
7. 좀 더 높은 성능의 하이퍼파라미터를 찾기 위해 반복적으로 탐색을 수행하도록 합니다. 이 때, GridSearchCV에 전달할 파라미터를 세분화하여 다시 교차검증을 수행하되, 별도의 코드셀을 사용하도록 합니다.

1. 데이터는 pandas의 read_csv() 메서드를 이용하여 winequality-red.csv 파일로부터 읽으면 됩니다.

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures  
from sklearn.preprocessing import StandardScaler
from scipy.stats import randint
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

In [2]:
wq_df = pd.read_csv('./winequality-red.csv')

In [3]:
wq_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [4]:
X  = wq_df.drop(columns='quality')
y  = wq_df['quality']

In [5]:
X 

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4
...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2


2. 데이터 전처리 시 PolyFeatures(2차항까지) StandardScaler를 사용하십시오.

In [6]:
X_train, X_test, y_train ,y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [7]:
# 2차 항을 추가하여 다항식을 생성.
poly_features = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly_features.fit_transform(X_train)
poly_features.get_feature_names()  # 생성된 특성명을 모두 확인

['x0',
 'x1',
 'x2',
 'x3',
 'x4',
 'x5',
 'x6',
 'x7',
 'x8',
 'x9',
 'x10',
 'x0^2',
 'x0 x1',
 'x0 x2',
 'x0 x3',
 'x0 x4',
 'x0 x5',
 'x0 x6',
 'x0 x7',
 'x0 x8',
 'x0 x9',
 'x0 x10',
 'x1^2',
 'x1 x2',
 'x1 x3',
 'x1 x4',
 'x1 x5',
 'x1 x6',
 'x1 x7',
 'x1 x8',
 'x1 x9',
 'x1 x10',
 'x2^2',
 'x2 x3',
 'x2 x4',
 'x2 x5',
 'x2 x6',
 'x2 x7',
 'x2 x8',
 'x2 x9',
 'x2 x10',
 'x3^2',
 'x3 x4',
 'x3 x5',
 'x3 x6',
 'x3 x7',
 'x3 x8',
 'x3 x9',
 'x3 x10',
 'x4^2',
 'x4 x5',
 'x4 x6',
 'x4 x7',
 'x4 x8',
 'x4 x9',
 'x4 x10',
 'x5^2',
 'x5 x6',
 'x5 x7',
 'x5 x8',
 'x5 x9',
 'x5 x10',
 'x6^2',
 'x6 x7',
 'x6 x8',
 'x6 x9',
 'x6 x10',
 'x7^2',
 'x7 x8',
 'x7 x9',
 'x7 x10',
 'x8^2',
 'x8 x9',
 'x8 x10',
 'x9^2',
 'x9 x10',
 'x10^2']

In [8]:
X_test_poly = poly_features.transform(X_test)

In [9]:
ss = StandardScaler()
X_train_pp = ss.fit_transform(X_train_poly)
X_test_pp = ss.transform(X_test_poly)

In [10]:
X_train_pp.max()

17.445589632467904

3. LogisticRegression을 이용해 와인의 품질을 예측하는 모델 객체를 생성하십시오.
4. GridSearchCV를 사용하여 교차검증을 하되, 최적의 모델을 찾기 위해 penalty, C의 값을 탐색하십시오.
5. 여기서 penalty는 L1, L2, elasticnet을 사용하고, elasticnet에 대해서는 l1_ratio 값을 함께 탐색해야 합니다.
6. 위의 세 가지 규제를 모두 수용하기 위해서 solver는 'saga'를 선택해야 하며, tol=0.001로 설정합니다.
7. 좀 더 높은 성능의 하이퍼파라미터를 찾기 위해 반복적으로 탐색을 수행하도록 합니다. 이 때, GridSearchCV에 전달할 파라미터를 세분화하여 다시 교차검증을 수행하되, 별도의 코드셀을 사용하도록 합니다.

In [11]:
# 5. 여기서 penalty는 L1, L2, elasticnet을 사용하고, elasticnet에 대해서는 l1_ratio 값을 함께 탐색해야 합니다.
# 6. 위의 세 가지 규제를 모두 수용하기 위해서 solver는 'saga'를 선택해야 하며, tol=0.001로 설정합니다.
# 7. 좀 더 높은 성능의 하이퍼파라미터를 찾기 위해 반복적으로 탐색을 수행하도록 합니다. 이 때, GridSearchCV에 전달할 파라미터를 세분화하여 다시 교차검증을 수행하되, 별도의 코드셀을 사용하도록 합니다.
param_grid = [ 
    # 12(=3×4)개의 하이퍼파라미터 조합을 시도합니다.
#     {'max_iter':[10000], 'multi_class':['multinomial'],'penalty': ['l1'], 'solver': ['saga'], 'tol':[0.0001], 'C':[0.001, 0.01, 0.1, 1, 10, 100]},
    {'max_iter':[10000], 'multi_class':['multinomial'],'penalty': ['l1','l2'], 'solver': ['saga'], 'tol':[0.0001], 'C':[0.001, 0.01, 0.1, 1, 10, 100]},
    {'max_iter':[10000],'multi_class':['multinomial'],'penalty': ['elasticnet'], 'l1_ratio': [0, 0.2, 0.4, 0.6, 0.8, 1], 'solver': ['saga'], 'tol':[0.0001], 'C':[0.001, 0.01, 0.1, 1, 10, 100]}
  ]


In [12]:
X_train_pp.shape

(1279, 77)

In [13]:
lr = LogisticRegression()
# lr.fit(X_train, y_train)

In [14]:
grid_svr_search = GridSearchCV(lr, param_grid
                           , cv=2
                           , scoring='accuracy'
                           , return_train_score=True)
grid_svr_search.fit(X_train_pp, y_train)

GridSearchCV(cv=2, estimator=LogisticRegression(),
             param_grid=[{'C': [0.001, 0.01, 0.1, 1, 10, 100],
                          'max_iter': [10000], 'multi_class': ['multinomial'],
                          'penalty': ['l1', 'l2'], 'solver': ['saga'],
                          'tol': [0.0001]},
                         {'C': [0.001, 0.01, 0.1, 1, 10, 100],
                          'l1_ratio': [0, 0.2, 0.4, 0.6, 0.8, 1],
                          'max_iter': [10000], 'multi_class': ['multinomial'],
                          'penalty': ['elasticnet'], 'solver': ['saga'],
                          'tol': [0.0001]}],
             return_train_score=True, scoring='accuracy')

In [16]:
grid_svr_search.best_params_

{'C': 1,
 'max_iter': 10000,
 'multi_class': 'multinomial',
 'penalty': 'l1',
 'solver': 'saga',
 'tol': 0.0001}